Magnetoencephalography (MEG)
----------

<table style="margin-left: 0px">
  <tr>
    <td>
      <img width=1000 height=200 src="MEG_experiment.PNG"/>
    </td>
   </tr>
</table>


In [1]:
import numpy as np
from IPython.display import display
import mne
mne.set_log_level('WARNING')

import bqplot.pyplot as plt

### Loading data using `MNE`

In [2]:
from mne.datasets import spm_face
data_path = spm_face.data_path()  # downloaded automatically (approx. 2GB)
raw_fname = data_path + '/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds'
print(raw_fname)

/Users/ccherukuri/mne_data/MNE-spm-face/MEG/spm/SPM_CTF_MEG_example_faces1_3D.ds


In [3]:
raw = mne.io.read_raw_ctf(raw_fname, preload=True)
print(raw)

<RawCTF  |  SPM_CTF_MEG_example_faces1_3D.meg4, n_channels x n_times : 340 x 324474 (676.0 sec), ~842.5 MB, data loaded>


In [4]:
raw.filter(1., 40., fir_design='firwin')

<RawCTF  |  SPM_CTF_MEG_example_faces1_3D.meg4, n_channels x n_times : 340 x 324474 (676.0 sec), ~842.5 MB, data loaded>

In [5]:
events = mne.find_events(raw, stim_channel='UPPT001', verbose=True)

print(events[:5])

172 events found
Events id: [1 2 3]
[[2026    0    3]
 [3759    0    3]
 [5484    0    3]
 [7226    0    3]
 [8951    0    1]]


In [6]:
event_id = {"faces": 1, "scrambled": 2}
tmin, tmax = -0.1, 0.5  # start and end of an epoch in sec.

# Set up indices of channels to include in analysis
picks = mne.pick_types(raw.info, meg=True, stim=True, eog=True,
                       ref_meg=True, exclude='bads')

# Read epochs
decim = 2  # decimate to make the example faster to run
epochs = mne.Epochs(raw, events, event_id, tmin, tmax, proj=True,
                    picks=picks, baseline=None, preload=True,
                    reject=dict(mag=1.5e-12), decim=decim)

print(epochs)

<Epochs  |  n_events : 167 (all good), tmin : -0.1 (s), tmax : 0.5 (s), baseline : None, ~57.1 MB, data loaded,
 'faces': 84, 'scrambled': 83>


In [7]:
# %matplotlib inline
evoked = epochs.average()
# evoked.plot_joint(times=[0.105, 0.130, 0.180]);

In [8]:
data = epochs.get_data()
data_mean = np.mean(data, axis=0)[31:]

In [9]:
# Color Utilities

def _rgb(x, y, z):
    """Transform x, y, z values into RGB colors."""
    rgb = np.array([x, y, z]).T
    rgb -= rgb.min(0)
    rgb /= rgb.max(0)
    return rgb

def rgb_to_css(rgb):
    '''
    Converts rgb color representation to a css hex
    '''
    for color in rgb:
        if color > 1 or color < 0:
            raise ValueError('rgb values must be between 0 and 1, a value of rgb({}) was given'.format(rgb))
    hexes = ["%02X" % int(color*255) for color in rgb]
    return ''.join(['#'] + hexes)


def color_scale(data, colors):
    n_colors = len(colors)
    mi, ma = data.min(), data.max()
    def f(x):
        index = int(n_colors * (x - mi) / (ma - mi))
        return colors[index - 1]
    return f

def diverging_color_scale(data, colors):
    mi, ma = data.min(), data.max()
    def f(x):
        if x < 0:
            t = (mi - x) / mi
            t = np.clip(t, 0, 1)
            return (1-t) * colors[0] + t * colors[1]
        else:
            t = (ma - x) / ma
            t = np.clip(t, 0, 1)
            return (1-t) * colors[2] + t * colors[1]
    return f

In [10]:
chs = evoked.info['chs']
locs3d = np.array([ch['loc'][:3] for ch in chs])[31:]
x, y, z = locs3d.T
colors = _rgb(x, y, z)
colors = [rgb_to_css(c) for c in colors]

In [11]:
from bqplot import ColorScale

In [12]:
fig2 = plt.figure(min_aspect_ratio=1, max_aspect_ratio=1, title='Seen from above',
                  layout={'min_width': '600px', 'min_height': '600px'})
sc_c = ColorScale(colors=['orangered', 'white', 'deepskyblue'], min=data_mean.min()* 1e15, max=data_mean.max()* 1e15)
scat = plt.scatter(x, y, colors=colors, scales={'color': sc_c},
                   axes_options={'x': {'visible': False}, 'y': {'visible': False}})
plt.show()

A Jupyter Widget

In [ ]:
def update_colors(name, value):
    sel = l.selected
    if len(sel) == 0:
        return
    scat.color = data_mean[:, sel[0]] * 1e15
    fig2.title = 'Snapshot at t = {:.1f} ms'.format(value[0])
    
fig = plt.figure(title='MEG signal', layout={'min_width': '800px'})

l = plt.plot(1e3 * epochs.times, 1e15 * data_mean,
         stroke_width=.2, interpolation='cardinal',
         colors=colors,
         axes_options={'x': {'label': 'Time (ms)'},
                       'y': {'label': 'Magnetic Field (fT)'}})
index_sel = plt.index_selector(update_colors)
plt.vline(0, line_style='dashed')

display(fig, fig2)

## How about 3d?

- pythreeJS https://github.com/jovyan/pythreejs
- ipyvolume https://github.com/maartenbreddels/ipyvolume

3d-plotting libraries based on widgets and WebGL

In [ ]:
from pythreejs import *
import numpy as np
from IPython.display import display

sensors = [Mesh(geometry=SphereGeometry(radius=0.003), 
                   material=LambertMaterial(color=colors[i]),
                   position=[x[i], y[i], z[i]])
              for i in range(len(x))]

scene = Scene(children=sensors + [AmbientLight(color='white')])

c = PerspectiveCamera(position=[0, 1, 1], up=[0, 0, 2],)

renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])
# display(renderer)

In [ ]:
red, black, blue = np.array([255, 69, 0])/255, np.array([1., 1., 1.]), np.array([0, 191, 255])/255
col_sc = diverging_color_scale(data_mean, [red, black, blue])
precomputed_colors = np.array([[rgb_to_css(col_sc(d)) for d in di] for di in data_mean])

def update_colors_3d(change):
    sel = l.selected
    if len(sel) == 0:
        return
    for i, b in enumerate(sensors):
        b.material.color = precomputed_colors[i, sel[0]]
        
fig = plt.figure(title='MEG signal', layout={'min_width': '800px'})

l = plt.plot(1e3 * epochs.times, 1e15 * data_mean,
         stroke_width=.2, interpolation='cardinal',
         colors=colors,
         axes_options={'x': {'label': 'Time (ms)'},
                       'y': {'label': 'Magnetic Field (fT)'}})
plt.index_selector(update_colors_3d)
plt.vline(0, line_style='dashed')

display(fig, renderer)